In [56]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Dependencies

In [4]:
from craigslist import CraigslistHousing
import timeit, os
from tqdm import tqdm
import pickle


import requests
from selenium import webdriver
from pymongo import MongoClient
import time
import pandas as pd
import missingno
import re
import numpy as np
# import json
# from pprint import pprint
from datetime import datetime, timedelta, timezone

from urls_list import * #where all urls and paths are saved
from config import *
# from harvesine_distance import harvesine_distance

In [58]:
## Craigslist provides a total of 3000 listings at once.

In [59]:
def craigs_list_api_call():
# This function returns the result of the listings based on site and category
# The search URL is as below
# https://toronto.craigslist.org/search/tor/apa?
    cl_tor_housing = CraigslistHousing(site='toronto',
                                       area='tor',
                                       category='apa',
                             filters={'bundle_duplicates': 1})

    #If geotagged=True, the results will include the (lat, lng) in the 'geotag' attrib (this will make the process a little bit longer).
    craiglist_housing = []

    start = timeit.timeit()
    for result in tqdm(cl_tor_housing.get_results(sort_by='newest', geotagged=True)):
        craiglist_housing.append(result)
    end = timeit.timeit()
    print(f"Time elapsed is {end - start}")
    return craiglist_housing

craiglist_housing = craigs_list_api_call()

3000it [19:15,  2.60it/s]

Time elapsed is 0.0001298019997193478


In [61]:
def differencer(DF):
    #Finds out the data to be scraped after comparing with the 
    #already existing data in the data base
    # Input - DF
    # Output - To be scraped IDs
    # Also clears the CurrentRental collection and updates it with the current rental
    all_postings_ids = DF['id'].map(lambda x: 'c_'+str(x))
    client = MongoClient(db_connection_string)
    db = client.ETLInsights #DB
    retrieved_postings = list(db.HistoricRental.find({'id': {'$in': list(all_postings_ids)}}))
    to_be_scraped_ids = set(all_postings_ids).difference(map(lambda x: x['id'], retrieved_postings))
    #Truncate and update the CurrentRental collection
    db.CurrentRental.delete_many({})
    db.CurrentRental.insert_many(retrieved_postings)
    client.close()
    return list(map(lambda x: x[2:], to_be_scraped_ids))

In [62]:
to_be_scraped_ids = differencer(pd.DataFrame(craiglist_housing))

In [63]:
# with open(os.path.join('..','Stage','craiglist_housing.pkl'), 'wb') as f:
#     pickle.dump(craiglist_housing, f)

# with open(os.path.join('..','Stage','craiglist_housing.pkl'), 'rb') as f:
#     craiglist_housing = pickle.load(f)

###  Scraping the craigslist posting data using the urls obtained through craigslist Python Module call

In [64]:
def instatiate_driver():
    #########################################################################################
    #Instatiate Selenium driver
    #Returns the handle object
    #########################################################################################
    chrome_options = webdriver.ChromeOptions()
    CHROMEDRIVER_PATH = executable_path
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')
    driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, options=chrome_options)
    return driver

In [65]:
def craigs_list_scrape(craigs_list_post_docs):
    #Instatiate the selenium driver
    driver = instatiate_driver()
    error_list = []
    

    craigs_list_post_docs_cp = craigs_list_post_docs.copy()
    for post in tqdm(craigs_list_post_docs_cp):

        #Visit the url
        driver.get(post['url'])

        #Separate Try Except to handle each cases separately
        #Apartment feature(Some extra feature beside the title)
        try:
            apartment_feature = driver.find_element_by_css_selector('span[class="postingtitletext"] span[class="housing"]').text
        except:
            #print(post['id'])
            error_list.append({post['id']:'apartment_feature'})
            apartment_feature = None
        finally:
            post['apartment_feature'] = apartment_feature

        #First image if present   
        try:
            image = None
            if post['has_image']:
                image = driver.find_element_by_css_selector('div[class="gallery"]').find_element_by_css_selector('img').get_attribute('src')
        except:
            print(post['id'])
            error_list.append({post['id']:'image'})
        finally:
            post['has_image'] = image

        #Body of the post  
        try:
            posting = driver.find_element_by_id("postingbody").text
        except:
            print(post['id'])
            error_list.append({post['id']:'postingbody'})
            posting = None
        finally:
            post['posting'] = posting

        #Attributes      
        try:
            attributes = [elem.text for elem in driver.find_elements_by_css_selector('p[class="attrgroup"]  span')]
        except:
            print(post['id'])
            error_list.append({post['id']:'attributes'})
            attributes = []
        finally:
            post['attributes'] = attributes
            
        time.sleep(1)
            
    driver.quit()       
    return craigs_list_post_docs_cp

In [66]:
#Filter out the to be scraped listings and scrape further
TBS = list(filter(lambda x: x['id'] in to_be_scraped_ids, craiglist_housing))
craiglist_housing_enriched = craigs_list_scrape(TBS)

100%|██████████| 150/150 [04:06<00:00,  1.64s/it]


# Transform

### Initial form of data after scraping

In [69]:
DF = pd.DataFrame(craiglist_housing_enriched)
DF.head()

,id,repost_of,name,url,datetime,last_updated,price,where,has_image,geotag,deleted,apartment_feature,posting,attributes
0,7201709212,None,2 Bedroom Basement – College and Dufferin,https://toronto.craigslist.org/tor/apa/d/west-...,2020-09-23 16:37,2020-09-23 16:37,"$1,485",College and Dufferin,https://images.craigslist.org/00202_29wIznhRoD...,"(43.6655, -79.4378)",False,/ 2br -,Brand new! Newly renovated 2 bedroom basement ...,"[2BR / 1Ba, available oct 1, house, laundry in..."
1,7201707943,None,Spacious 1 Bedroom Condo With Breathtaking Vie...,https://toronto.craigslist.org/tor/apa/d/toron...,2020-09-23 16:35,2020-09-23 16:35,"$1,800",Toronto,https://images.craigslist.org/00Q0Q_gHdEJXzZSW...,"(43.652648, -79.358122)",False,/ 1br -,Call or Text Alex at show contact info to book...,"[1BR / 1Ba, available dec 1, condo, w/d in uni..."
2,7201706377,None,Newly Renovated 2 bedroom in Annex,https://toronto.craigslist.org/tor/apa/d/downt...,2020-09-23 16:32,2020-09-23 16:32,"$2,550",Bloor and Spadina,https://images.craigslist.org/00z0z_icZss0ICCe...,"(43.6629, -79.3987)",False,/ 2br -,Newly renovated 2 bedroom located in the prest...,"[2BR / 1Ba, available oct 1, house, w/d in uni..."
3,7201703978,None,3BR/3BA EndUnit Townhouse w/backyard inc.water...,https://toronto.craigslist.org/tor/apa/d/north...,2020-09-23 16:29,2020-09-23 16:29,"$3,600","Willowdale, Toronto",https://images.craigslist.org/00p0p_jNg8NSL8xM...,"(43.7673, -79.4111)",False,/ 3br - 1915ft2 -,Extra-wide end-unit executive townhouse in sou...,"[3BR / 2.5Ba, 1915ft2, available nov 1, EV cha..."
4,7201703853,None,Renovated 2 bedroom at Christie & Bloor,https://toronto.craigslist.org/tor/apa/d/downt...,2020-09-23 16:28,2020-09-23 16:28,"$2,350",Christie & Bloor,https://images.craigslist.org/00e0e_hTyQbkdE5Y...,"(43.6683, -79.4205)",False,/ 2br -,Newly renovated 2 bedroom on the main floor at...,"[2BR / 1Ba, available oct 1, house, laundry in..."


### Remove Duplicates and unreliable data
- All duplicated data are removed
- Dont consider empty postings
- Drop all those rows which don't have both 'geotag' and 'where'
- Convert attributes list to string
- Convert the apartment feature none to ''(string)

In [70]:
def clean_craigslist(DF):
    ### Remove Duplicates and unreliable data
    # - All duplicated data are removed
    # - Dont consider empty postings
    # - Drop all those rows which don't have both 'geotag' and 'where'
    # Convert attributes list to string
    # Convert the apartment feature none to ''(string)

    #Duplicates cannot hash list
    DF.drop_duplicates(subset=DF.columns[:-1], keep="first", inplace=True)
    #Dont consider empty postings
    DF.dropna(subset=['posting'], inplace=True)
    #Location:- Coming to whereabouts, in most of the cases, we have geotag. Some of the cases which doesn't have geotag,
    #we have "where". Check below.We need to drop all those rows which don't have both
    DF.dropna(subset=['where', 'geotag'], how='all', inplace=True)
    #Convert attributes list to string
    DF['attributes'] = DF['attributes'].map(lambda x: ' '.join(x))
    #Convert the apartment feature none to ''(string)
    DF['apartment_feature'].fillna(value=' ', inplace=True)#A string val
    #Combine the "attributes", "apartment_feature", "posting" together as "text" column
    DF["text"] = DF.apply(lambda x: x["attributes"]+x["apartment_feature"]+x["posting"], axis=1)
    #Use DF["text"] for all further extractions 
    return DF


In [71]:
DF = clean_craigslist(pd.DataFrame(craiglist_housing_enriched))

## Deive new Features
- Combine the "attributes", "apartment_feature", "posting" together as "text" column for string search using regex to derive new features
- New features derived are
    - sf - Square Feet : (int or None) Based on ft2|SqFt in the text data
    - br - Bed Room: (float or None) Number of bedrooms, based on what is preceding BR 
    - ba - Bath: (float or None) Number of bath, based on what is preceding Ba
    - cats_allowed - True or False, based on the presence of 'cats are OK - purrr'
    - dogs are OK - wooof - True or False, based on the presence of 'dogs are OK - wooof'
    - Type - Type of the housing 1 out of ``['condo', 'house', 'apartment', 'suite', 'townhouse', 'loft', 'duplex',
       'flat', 'cottage', 'land']`` or ``None``
    - furnished - True/Flase/None - based on the presense of (un|non)(-) furnished strings or nothing
    

```unique_attributes = set()
for entry in DF['attributes']:
    unique_attributes = unique_attributes.union(set(entry))
unique_attributes```  

In [72]:
def extract(s):
    #DF['Text'] comes here as the input
    #Square Feet
    sf_found = re.findall('(\d+) *(ft2|SqFt)', s)
    sf =  (None if not sf_found else int(sf_found[0][0]))

    #Bed Room
    br_found = re.findall('(\d){0,1}(.5){0,1} *BR', s)
    br = float(''.join(br_found[0])) if  br_found and ''.join(br_found[0]) else None #To handle ('','','') situation
            
    
    #Bath
    ba_found = re.findall('(\d){0,1}(.5){0,1}(\+){0,1} *B[aA]',s)
    ba = float(re.sub('[^\d]', '', ''.join(ba_found[0]))) if  ba_found and re.sub('[^\d]', '', ''.join(ba_found[0])) else None
    
    
    #'cats are OK - purrr'
    cats_allowed = (True if re.findall('cats are OK - purrr', s) else False)
    
    #'dogs are OK - wooof'
    dogs_allowed = (True if re.findall('dogs are OK - wooof', s) else False)
    
    #Check the type of the commodity
    re.sub(r'[\n\.,!/?()]', ' ', s.lower()) #Remove unnecessary chars. We need to capture apartment\ also as word apartment
    cleaned_s = re.sub('[\n\\.,!\/?]', ' ', s)
    found_type = re.findall(r"\b(townhouse|loft|land|house|flat|duplex|condo|cottage|suite)\b", cleaned_s, flags=re.IGNORECASE) 
    Type = (None if not found_type else found_type[0].lower())
    #Though apartment is a generic term, some people mention the type as apartment, we are going to take that as the last priority
    #if nothing else is mentioned
    if not Type:
        found_type = re.findall(r"\bapartment\b", cleaned_s, flags=re.IGNORECASE)
        Type = (None if not found_type else found_type[0].lower())
        
    #Furnished or Unfurnished checks
    found_un = re.findall('(non|un)-*(?=furnished)', s , flags=re.IGNORECASE)
    furnished = False
    if not found_un:
        #furnished = None #Nothing found
        found_furnished = re.findall('furnished', s , flags=re.IGNORECASE)
        furnished = (None if not found_furnished else True)
    
    return [sf,br,ba, cats_allowed, dogs_allowed, Type, furnished]

In [73]:
DF[['sf', 'BR', 'Ba','cats_allowed', 'dogs_allowed', 'Type', 'furnished']] = pd.DataFrame(DF['text'].map(lambda x : extract(x)).to_list(), index=DF.index)

### Geocode the address to lat long

In [74]:
#Dictionary with index and address
def geocode(addresses):
    #Input: a dictionary with key as index and value as address
    #Output: a dictionary with key as index and value as (lat, long) tuple
    lat_long = {}
    for index in addresses:
        try:
            url = geocode_api+f'?address={addresses[index]}'+f'&key={google_api}'
            url = re.sub(' +', '%20', url)
            response = requests.get(url)
            if response.ok:
                content = response.json()
                lat_long[index]=tuple(content['results'][0]['geometry']['location'].values())
            else:
                lat_long[index]=None
        except Exception as e:
            print(e)
            lat_long[index]=None
    return lat_long

In [75]:
#Dictionary with index and address
def geocodeMQ(addresses):
    #Input: a dictionary with key as index and value as address
    #Output: a dictionary with key as index and value as (lat, long) tuple
    lat_long = {}
    for index in addresses:
        try:
            #Small tweek for 'queens quay'
            location = (addresses[index]+' ,Toronto' if addresses[index].lower()=='queens quay' else addresses[index])
            url = geocode_mq_base+f'key={mq_key}&location={location}'
            url = re.sub(' +', '%20', url)
            response = requests.get(url)
            if response.ok:
                content = response.json()
                lat_long[index]=tuple(content['results'][0]['locations'][0]['latLng'].values())
            else:
                lat_long[index]=None
        except Exception as e:
            print(e)
            lat_long[index]=None
    return lat_long

In [76]:
addresses = DF[DF['geotag'].isnull()]['where'].to_dict()

In [77]:
lat_long = geocodeMQ(addresses)
## Replace the Null geocodes with the geocodes retrieved from the address
DF.loc[lat_long.keys(), 'geotag'] = DF.loc[lat_long.keys()].index.map(lat_long)

In [78]:
#Now no missing values in geocode
#Check for nan
missingdata_cols = DF.columns[DF.isnull().any()].to_list()
missingdata_cols

['repost_of', 'where', 'has_image', 'sf', 'BR', 'Ba', 'furnished']

### Postal codes based on lat long

In [79]:
#https://maps.googleapis.com/maps/api/geocode/json?latlng=40.714224,-73.961452&key=YOUR_API_KEY
#reverse_geocode_api = 'https://maps.googleapis.com/maps/api/geocode/json'

#Dictionary with index and address
def reverse_geocode(loc):
    #Input: tuple - (lat, long)
    #Output: string - Postal Code
    lat, long = loc
    postal_code = None
    try:
        url = reverse_geocode_api+f'?latlng={lat},{long}&result_type=postal_code'+f'&key={google_api}'
        url = re.sub(' +', '%20', url)
        #print(url)
        response = requests.get(url)
        if response.ok:
            content = response.json()
            postal_code = content['results'][0]["address_components"][0]['long_name'] if 'postal_code' in content['results'][0]\
["address_components"][0]['types'] else None
    except Exception as e:
        print(e)
    return postal_code   

In [80]:
#Dictionary with index and address
def reverse_geocodeMQ(loc):
    #Input: tuple - (lat, long)
    #Output: string - Postal Code
    lat, long = loc
    postal_code = None
    try:
        url = reverse_geocode_mq_base.format(mq_key, lat, long)
        url = re.sub(' +', '%20', url)
        #print(url)
        response = requests.get(url)
        if response.ok:
            content = response.json()
            postal_code = content['results'][0]["locations"][0]['postalCode'] if 'postalCode'\
            in content['results'][0]["locations"][0] else None
    except Exception as e:
        print(e)
    return postal_code  

In [81]:
DF['postal_code'] = DF['geotag'].map(lambda x: reverse_geocodeMQ(x))
DF[DF['postal_code'].notnull()].shape
DF.shape
# Discarding those 6 items which doesn't have postal code
DF.drop(index=DF[DF['postal_code'].isnull()].index, inplace=True)

(147, 23)

(147, 23)

``DF['postal_code'] = DF['geotag'].map(lambda x: reverse_geocode(x))``

``DF[DF['postal_code'].notnull()].shape``

``DF.shape``

``#Discarding those 7 items which doesn't have postal code``

``DF.drop(index=DF[DF['postal_code'].isnull()].index, inplace=True)``

In [82]:
Florida_Zipcodes = DF['postal_code'][DF['postal_code'].map(lambda x : re.findall('^[\d].*', x)[0] if re.findall('^[\d].*', x) else None).notnull()].index 


In [83]:
DF.drop(index=Florida_Zipcodes, inplace=True)

### Save standalone craigslist dataset

In [54]:
with open(os.path.join('..','Stage','craiglist_standalone.pkl'), 'wb') as f:
    pickle.dump(DF, f)

In [55]:
with open(os.path.join('..','Stage','craiglist_standalone.pkl'), 'rb') as f:
    DF = pickle.load(f)

### Tranformation required to append the craigslist table with kijii table

- **``We need to concatenate craigslist table with kijiji table``**
- **``All the column names have to be unique``**

In [84]:
def clean_rental_for_merge(df):
    DF= df.copy()
    #To rename
    DF.rename(columns={"has_image":"image", "name":"title", "datetime":"post_published_date", "where":"address", "Type":"rental_type", "BR":"bedrooms", "Ba":"bathrooms", "sf":"sqft", "text":"description"}, inplace=True)              
    #To extract
    DF['pet_friendly'] = DF.apply(lambda x: x['cats_allowed'] | x['dogs_allowed'], axis=1)
    DF['lat'] = DF.geotag.map(lambda x: float(x[0]))
    DF['long'] = DF.geotag.map(lambda x: float(x[1]))
    #To clean
    DF['price'] = DF['price'].map(lambda x: re.sub('[^\d]', '',x))
    #To drop
    DF.drop(["repost_of", "deleted", "apartment_feature", "cats_allowed", "dogs_allowed", "geotag", "posting", "attributes", "last_updated"], axis=1, inplace=True)
    #Add one source column
    DF['source'] = 'craigslist'
    #FSA
    DF['FSA']=DF['postal_code'].map(lambda x:x.split(' ')[0])
    #Reorder
    DF = DF[['id', 'title', 'price', 'sqft','image','url','post_published_date', 'lat', 'long', 'postal_code', 'FSA', 'rental_type','bedrooms', 'bathrooms', 'furnished', 'pet_friendly', 'description', 'source']]
    #Append c_ to the index
    DF['id'] = DF['id'].map(lambda x: 'c_'+str(x))
    #Change date
    DF.post_published_date = DF.post_published_date.map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M").strftime("%Y-%m-%d"))
    
    return DF
    

In [85]:
DF = clean_rental_for_merge(DF)

In [86]:
DF

,id,title,price,sqft,image,url,post_published_date,lat,long,postal_code,FSA,rental_type,bedrooms,bathrooms,furnished,pet_friendly,description,source
0,c_7201709212,2 Bedroom Basement – College and Dufferin,1485,NaN,https://images.craigslist.org/00202_29wIznhRoD...,https://toronto.craigslist.org/tor/apa/d/west-...,2020-09-23,43.665500,-79.437800,M6H 4B9,M6H,house,2.0,1.0,None,False,2BR / 1Ba available oct 1 house laundry in bld...,craigslist
1,c_7201707943,Spacious 1 Bedroom Condo With Breathtaking Vie...,1800,NaN,https://images.craigslist.org/00Q0Q_gHdEJXzZSW...,https://toronto.craigslist.org/tor/apa/d/toron...,2020-09-23,43.652648,-79.358122,M5A,M5A,condo,1.0,1.0,None,False,1BR / 1Ba available dec 1 condo w/d in unit no...,craigslist
2,c_7201706377,Newly Renovated 2 bedroom in Annex,2550,NaN,https://images.craigslist.org/00z0z_icZss0ICCe...,https://toronto.craigslist.org/tor/apa/d/downt...,2020-09-23,43.662900,-79.398700,M5S 2E8,M5S,house,2.0,1.0,None,False,2BR / 1Ba available oct 1 house w/d in unit no...,craigslist
3,c_7201703978,3BR/3BA EndUnit Townhouse w/backyard inc.water...,3600,1915.0,https://images.craigslist.org/00p0p_jNg8NSL8xM...,https://toronto.craigslist.org/tor/apa/d/north...,2020-09-23,43.767300,-79.411100,M2N,M2N,townhouse,3.0,25.0,None,True,3BR / 2.5Ba 1915ft2 available nov 1 EV chargin...,craigslist
4,c_7201703853,Renovated 2 bedroom at Christie & Bloor,2350,NaN,https://images.craigslist.org/00e0e_hTyQbkdE5Y...,https://toronto.craigslist.org/tor/apa/d/downt...,2020-09-23,43.668300,-79.420500,M6G 3B5,M6G,house,2.0,1.0,None,False,2BR / 1Ba available oct 1 house laundry in bld...,craigslist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,c_7198592543,Spacious 2 bedrooms apartment,1539,800.0,https://images.craigslist.org/00h0h_8yw9dpRRV7...,https://toronto.craigslist.org/tor/apa/d/spaci...,2020-09-18,-40.501994,-65.126953,,,apartment,2.0,1.0,False,False,2BR / 1Ba 800ft2 available oct 1 apartment w/d...,craigslist
146,c_7189528276,"Stunning 2-Bedroom, 1 Washroom Unit on Univers...",2500,NaN,https://images.craigslist.org/00909_6rLSGsNzEo...,https://toronto.craigslist.org/tor/apa/d/midhu...,2020-09-15,43.654468,-79.388306,M5G 2N9,M5G,apartment,2.0,1.0,None,False,2BR / 1Ba apartment laundry on site no smoking...,craigslist
147,c_7195735830,"1 BDRM MAIN FLOOR, LARGE LIVING ROOM, HUGE CLO...",1650,NaN,https://images.craigslist.org/00707_jDJeKKnkOn...,https://toronto.craigslist.org/tor/apa/d/west-...,2020-09-13,43.665500,-79.437800,M6H 4B9,M6H,apartment,1.0,1.0,None,False,1BR / 1Ba available oct 1 apartment/ 1br -10 M...,craigslist
148,c_7195736550,"HUGE BACHELOR-FULL KITCHEN & 4 PIECE BTHRM, HU...",1250,NaN,https://images.craigslist.org/01717_hFWgXpd1E3...,https://toronto.craigslist.org/tor/apa/d/west-...,2020-09-13,43.665500,-79.437800,M6H 4B9,M6H,apartment,0.0,1.0,None,False,0BR / 1Ba available oct 1 apartment LARGE BACH...,craigslist


## Save all dataframes as csv

In [63]:
DF.to_csv(os.path.join('..','Output','Rental_Craigslist.csv'), index=False)

In [64]:
DF

,id,title,price,sqft,image,url,post_published_date,lat,long,postal_code,FSA,rental_type,bedrooms,bathrooms,furnished,pet_friendly,description,source
0,c_7200612095,1 BR condo at 68 Abell St--Queen West-- Availa...,1600,420.0,https://images.craigslist.org/00J0J_8zBmjxmDi0...,https://toronto.craigslist.org/tor/apa/d/toron...,2020-09-21,43.642777,-79.424243,M6J,M6J,condo,1.0,1.0,None,False,1BR / 1Ba 420ft2 available oct 1 condo w/d in ...,craigslist
1,c_7187385635,ALL INCLUSIVE 2 bedroom Furnished -Main Subway...,1595,NaN,https://images.craigslist.org/01010_fAb2gD7xgA...,https://toronto.craigslist.org/tor/apa/d/east-...,2020-09-21,43.693118,-79.297176,M4C 5B9,M4C,house,2.0,1.0,True,False,2BR / 1Ba available oct 1 furnished house w/d ...,craigslist
2,c_7200606976,Beautiful Rooms for rent in North York (immedi...,600,NaN,https://images.craigslist.org/00x0x_1wSiPFUT3f...,https://toronto.craigslist.org/tor/apa/d/north...,2020-09-21,43.804775,-79.380681,M2M 2Z5,M2M,house,4.0,3.0,True,False,4BR / 3Ba available now furnished house laundr...,craigslist
3,c_7200610375,GORGEOUS RENOVATED-2 BEDROOM UNIT- QUEENSWAY &...,2100,950.0,https://images.craigslist.org/00B0B_83hUDfsiTP...,https://toronto.craigslist.org/tor/apa/d/etobi...,2020-09-21,43.633793,-79.487718,M8Y 1R8,M8Y,suite,2.0,1.0,None,True,2BR / 1Ba 950ft2 available oct 1 cats are OK -...,craigslist
4,c_7200602760,Basement Apartment,750,NaN,https://images.craigslist.org/00j0j_gntmBwGe9O...,https://toronto.craigslist.org/tor/apa/d/richm...,2020-09-21,43.866900,-79.441400,L4C 9S5,L4C,apartment,NaN,NaN,None,False,available sep 25 apartment w/d in unit A beaut...,craigslist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,c_7175105221,"1BDR Condo above Loblaws, Queen/Portland",2200,650.0,https://images.craigslist.org/00t0t_gyX609atyM...,https://toronto.craigslist.org/tor/apa/d/toron...,2020-09-05,43.647085,-79.401455,M5V 1Y9,M5V,condo,1.0,1.0,None,False,1BR / 1Ba 650ft2 available oct 1 condo w/d in ...,craigslist
2996,c_7190663212,"Bloor and Spadina ANNEX, Downtown, mins to U o...",2700,NaN,https://images.craigslist.org/00808_8JF9DKqTva...,https://toronto.craigslist.org/tor/apa/d/toron...,2020-09-05,43.665900,-79.405425,M5S 2K8,M5S,house,3.0,2.0,None,False,3BR / 2Ba apartment w/d in unit no smoking no ...,craigslist
2997,c_7188127944,Bright Condo for Rent - 1Bd + Den + 1.5 Br in ...,2220,665.0,https://images.craigslist.org/00s0s_3MjkAuvQtj...,https://toronto.craigslist.org/tor/apa/d/toron...,2020-09-05,43.654895,-79.378975,M5B 1T8,M5B,condo,1.0,15.0,None,False,1BR / 1.5Ba 665ft2 available now condo w/d in ...,craigslist
2998,c_7173283683,1 Bed & 1 Bath + Parking in Yorkville,1999,NaN,https://images.craigslist.org/00v0v_hFIddf4dId...,https://toronto.craigslist.org/tor/apa/d/toron...,2020-09-05,43.672503,-79.388764,M5R 1E9,M5R,condo,1.0,1.0,None,False,1BR / 1Ba available oct 1 condo w/d in unit no...,craigslist


# Load

In [87]:
from pymongo import MongoClient

In [88]:
# client = MongoClient(db_connection_string)
# client.ETLInsights.HistoricRental.insert_many(DF.T.to_dict().values())
# client.ETLInsights.CurrentRental.insert_many(DF.T.to_dict().values())
# client.close()